## ハンズオン01: LLM アプリケーション Trace

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

ハンズオンに必要な環境変数を読み込みます。

In [1]:
import os

endpoint = "http://langfuse-server:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

Langfuseのクライアントを初期化します。

In [4]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

### 出力に対する評価

LLM as a Judgeの対象となる生成結果の一覧を取得します。今回は、現在時から24時間以内に生成された生成結果を評価対象として扱います。

In [ ]:
import datetime
from pprint import pprint

generations = langfuse.get_generations(
    from_start_time=datetime.datetime.now() - datetime.timedelta(hours=24),
)

pprint(f"Fetched {len(generations.data)} generations.")
pprint(f"{generations.data[0].__dict__}")

評価用の関数を実装します。今回は、LangChainのEvaluatorを使用します。

In [6]:
from langchain.evaluation.loading import load_evaluator
from langchain.evaluation.schema import EvaluatorType

def load_evaluator_by_criteria_key(key: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")

    evaluator = load_evaluator(
        evaluator=EvaluatorType.CRITERIA,
        llm=llm,
        criteria=key
    )
    return evaluator

評価基準を設定します。今回は、

- conciseness: 簡潔で要点をついた回答であるか
- coherence: 構造化され、整理された回答であるか
- harmfulness: 有害、攻撃的、不適切な回答であるか

を評価基準として設定します。

In [7]:
criterias = [
    "conciseness",
    "coherence",
    "harmfulness",
]

24時間以内の生成結果に対して、実際にLLMによる評価を行います。

In [ ]:
def execute_evaluation_and_scoring():
    for generation in generations.data:
        for key in criterias:
            evaluator = load_evaluator_by_criteria_key(key=key)
            result = evaluator.evaluate_strings(
                prediction=generation.output,
                input=generation.input
            )
            pprint(result)
            langfuse.score(
                name=f"llm-as-a-judge-{key}",
                trace_id=generation.trace_id,
                observation_id=generation.id,
                value=result.get("score"),
                comment=result.get("reasoning")
            )

execute_evaluation_and_scoring()

### （オプション）入力に対する評価

LLM as a Judgeの対象となる一覧を取得します。今回は、現在時から24時間以内に入力されたプロンプトを対象として扱います。

In [ ]:
import datetime
from pprint import pprint

traces = langfuse.get_traces(
    from_timestamp=datetime.datetime.now() - datetime.timedelta(hours=24),
    tags=["app"]
)

pprint(f"Fetched {len(traces.data)} generations.")
pprint(f"{traces.data[0].__dict__}")

評価用の関数を実装します。今回は、ユーザーの入力プロンプトを”否定的”、”中立的”、”肯定的”にLLMを用いて分類を行います。

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

fallback_prompt = """
以下の入力テキストを”否定的”、”中立的”、”肯定的”に分類してください。
また、出力は”否定的”、”中立的”、”肯定的”のみで理由などは含まないでください。

## 入力テキスト

{{input}}
"""

def sentiment_analysis(input: str) -> str:
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")
    prompt = langfuse.get_prompt(name="sentiment-analysis-prompt", fallback=fallback_prompt)
    sentiment_analysis_chain = (
        {"input": RunnablePassthrough()}
        | PromptTemplate.from_template(prompt.get_langchain_prompt())
        | llm
        | StrOutputParser()
    )
    result = sentiment_analysis_chain.invoke(input)
    return result

入力プロンプトに対する感情分析を実行します。

In [ ]:
def execute_sentiment_analysis():
    for trace in traces.data:
        result = sentiment_analysis(input=trace.input)
        score_map = {
            "否定的": 0,
            "中立的": 0.5,
            "肯定的": 1
        }
        pprint({"input": trace.input, "result": result})
        langfuse.score(
            name=f"llm-as-a-judge-sentiment-analysis",
            trace_id=trace.id,
            observation_id=trace.id,
            value=score_map.get(result, 0.5),
            comment=result
        )

execute_sentiment_analysis()